In [1]:
import pandas as pd
import re
import contractions
import spacy

from spacy.matcher import PhraseMatcher

In [6]:
df = pd.read_csv("../transcripts/mrs-r.csv")

df = df[df["Transcript"].str.startswith("Mrs")]
df["Transcript"] = df["Transcript"].apply(lambda x: re.sub(r"Mrs\.?\s*R\:?\s*", "", x))
df["Transcript"] = df["Transcript"].apply(lambda x: contractions.fix(x))
df["Transcript"] = df["Transcript"].apply(lambda x: x.lower())

transcript = " ".join(df["Transcript"])


In [22]:
nlp = spacy.load("en_core_web_sm") # tagger, (dependency) parser, ner
doc = nlp(transcript)
# print(nlp.pipe_names)

sentences = pd.Series([s.text for s in doc.sents])
# sentences = [s for s in doc.sents]

matcher = PhraseMatcher(nlp.vocab) # vocabulary object shared with the documents the matcher will operate on
pronouns = ["i", "me", "mine", "myself"]
patterns = [nlp(t) for t in pronouns]
matcher.add("FIRST PERSON PRONOUNS", patterns) # add key and list of Doc objects of the pattern

matches = matcher(doc)

first_person_sentences = []

for sent in doc.sents:
    for match_id, start, end in matcher(nlp(sent.text)):
        if nlp.vocab.strings[match_id] == "FIRST PERSON PRONOUNS":
            first_person_sentences.append(sent.text)

# print(nlp.vocab.strings[5581825994029504612]) # maps to FIRST PERSON PRONOUNS

In [28]:
first_person_sentences[:10]

['first of all, i still have a strong voice at the age of 90, so a lot of people on the phone do not think i am 90, but this year i am 90 and i have lived in santa clara valley, which is now called silicon valley, oh, most of my life.',
 'first of all, i still have a strong voice at the age of 90, so a lot of people on the phone do not think i am 90, but this year i am 90 and i have lived in santa clara valley, which is now called silicon valley, oh, most of my life.',
 'first of all, i still have a strong voice at the age of 90, so a lot of people on the phone do not think i am 90, but this year i am 90 and i have lived in santa clara valley, which is now called silicon valley, oh, most of my life.',
 'first of all, i still have a strong voice at the age of 90, so a lot of people on the phone do not think i am 90, but this year i am 90 and i have lived in santa clara valley, which is now called silicon valley, oh, most of my life.',
 'so that is where i grew up',
 ', i grew up in a to

In [30]:
list(set(first_person_sentences))[:20]

['i love tapioca pudding and i have not made it for a long time.',
 'and i would have taken airplane flights, but they were not available when i was that age.',
 'so i think the things we can create are just very, very important.',
 'having a large family, so we ended up having a large family, and now i have a lot of my daughters and my sons.',
 'i had wonderful times raising our children.',
 'so , i guess he became the attorney',
 'i was, i was born 1930, so just before the second world war started.',
 'in fact, a couple came over yesterday and brought me some of the couple slices of her bread.',
 'so i think getting, having time to grow up and be kids is very, very important.',
 'it is a very simple way of looking at life and every, every day i think about one of the agreements about how it is affecting me and how i can interact with other people with more kindness and listening.',
 'and i was bored.',
 'what actually happened to me, i, when i started working, i told my husband, i am

In [3]:
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("OBAMA", [nlp("Barack Obama")])
doc = nlp("Barack Obama lifts America one last time in emotional farewell")
matches = matcher(doc)

In [4]:
matches

[(7732777389095836264, 0, 2)]